In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
student_data = [("Chris",1523,0.72,"CA"),
                ("Jake", 1555,0.83,"NY"),
                ("Cody", 1439,0.92,"CA"),
                ("Lisa",1442,0.81,"FL"),
                ("Daniel",1600,0.88,"TX"),
                ("Kelvin",1382,0.99,"FL"),
                ("Nancy",1442,0.74,"TX"),
                ("Pavel",1599,0.82,"NY"),
                ("Josh",1482,0.78,"CA"),
                ("Cynthia",1582,0.94,"CA")]
student_rdd = spark.sparkContext.parallelize(student_data)
rdd_transformation = student_rdd.map(lambda x: (x[0], x[1], int(x[2]*100), x[3]))
states = {"NY":"New York", "CA":"California", "TX":"Texas", "FL":"Florida"}
broadcastStates = spark.sparkContext.broadcast(states)
rdd_broadcast = rdd_transformation.map(lambda x: (x[0],x[1],x[2],broadcastStates.value[x[3]]))

1. Create the accumulator variable that starts at 0 and name it `sat_1500`.

In [2]:
# create a spark accumulator variable that will track the no. of
# sat scores which are over 1500. this should start at 0
sat_1500 = spark.sparkContext.accumulator(0)

# confirm type
type(sat_1500)

pyspark.accumulators.Accumulator

2. Create a function called `count_high_sat_score` that increments our accumulator by 1 whenever it encounters a score of over 1500.

In [3]:
def count_high_sat_score(r):
    # these methods take the row as an argument
    # spark will iterate this method over all the rows
    if r[1] > 1500: sat_1500.add(1)


# confirm saved as a function
print(count_high_sat_score)

<function count_high_sat_score at 0x7fef2c9e90d0>


3. Call `count_high_sat_score` in an action that will apply the function to each element in `rdd_broadcast`.

In [4]:
# now implement this on our spark dataset
# we're not transforming anything here
# use foreach. this is basically a void
rdd_broadcast.foreach(lambda x: count_high_sat_score(x))


# confirm accumulator worked
print(sat_1500)

5
